# alpha prolog


In [1]:
! aprolog --help

          ______                  
          ||   \\    /|          
          ||    )    ||           
    __  _ ||___/_ __ ||  __   __  _
   / \\/  || \\/ / \\|| / \\ / \\/
  (   )\  ||  ||(   )||(   )(   ) 
  \\_/ \\_||  ||\\_/ ||\\_/ \\_/  
          /   /      /      ((__ 
                            \\__))  
AlphaProlog 0.4
For help, type "#help."
usage: aprolog [options] files
  -b Depth-first search bound
  -d Enable debugging messages
  -hh Permit hereditary Harrop programs (experimental)
  -ng Restrict to new-goal Horn clauses
  -s Simplify generated clauses
  -l Linearize clauses
  -wq Normalize and well-quantify clauses
  -skip-occurs Skip occurs check
  -q Quit interpreting after reading all input
  -L Add directory to library search path
  -so Strict name occurs checks (experimental)
  -susp Apply permutations as suspensions (experimental)
  -t Set tracelevel (0,1,2)
  -to Typecheck only
  -v Verbose mode
  -log Log mode
  -ap Allow prop as type
  -aho Allow higher-order 

In [2]:
%%file /tmp/esubst.apl
exp	: type.
var	: type.
sub	: type.

lam	: exp -> exp.
app	: exp -> exp -> exp.
one	: exp.
subst	: exp -> sub -> exp.

id	: sub.
lift	: sub.
cons	: exp -> sub -> sub.
comp	: sub -> sub -> sub.


pred 	eq	exp exp.
pred 	eq'	sub sub.

eq (app (lam A) B) (subst A (cons B id)).
eq (subst one id) one.
eq (subst one (cons A S)) A.
eq (subst (app B A) S) (app (subst B S) (subst A S)).
eq (subst (lam A) S) (lam (subst A (cons one (comp S lift)))).
eq (subst A (comp S T)) (subst (subst A S) T).

eq' (comp id S) S.
eq' (comp lift id) lift.
eq' (comp lift (cons A S)) S.
eq' (comp (cons A S) T) (cons (subst A T) (comp S T)).
eq' (comp (comp S T) U) (comp S (comp T U)).

pred 	step	exp exp.
pred	step'	sub sub.

step E E' :- eq E E'.
step (app E1 E2) (app E1' E2) :- step E1 E1'.
step (app E1 E2) (app E1 E2') :- step E2 E2'.
step (subst E S) (subst E' S) :- step E E'.
step (subst E S) (subst E S') :- step' S S'.

step' S S' :- eq' S S'.

?step (app (lam (lam one)) (lam one)) E.

Writing /tmp/esubst.apl


In [4]:
! aprolog /tmp/esubst.apl -q

          ______                  
          ||   \\    /|          
          ||    )    ||           
    __  _ ||___/_ __ ||  __   __  _
   / \\/  || \\/ / \\|| / \\ / \\/
  (   )\  ||  ||(   )||(   )(   ) 
  \\_/ \\_||  ||\\_/ ||\\_/ \\_/  
          /   /      /      ((__ 
                            \\__))  
AlphaProlog 0.4
For help, type "#help."
Reading file /tmp/esubst.apl...
--------
Query: step(app(lam(lam(one)))(lam(one)))(E)
Yes.
E = subst(lam(one))(cons(lam(one))(id))


In [32]:
%%file /tmp/alice.pl
kind person  type.
type mallory, bob, alice  person.
pred age o:person, o:int.

age mallory 23.
age bob 23.
age alice 20.

main :- age X Y, print X, print Y, fail.

Overwriting /tmp/alice.pl


In [33]:
! elpi -test /tmp/alice.pl


Parsing time: 0.000

Compilation time: 0.000

Typechecking time: 0.003
Determinacy checking time: 0.004
mallory
23
bob
23
alice
20




# Nominal logic
See knuckledragger
See slotted

idea: use dif/2 constraints + var(X) guarded to do Rudi style slotted as unification variables.
Maybe an interpreter that carries all atoms in context.
Freshness constraints?


act(Perm) and apply(Subst) are very similar creatures
triangular permutations and triangular substitutions

⟨a⟩t = ⟨b⟩u ⇒ t = (ab)⋅u  and  a # u
Yes, this is the alpha equivalence check

https://www.cl.cam.ac.uk/~amp12/papers/nomu/nomu.pdf Pitts gabbay 2004


Miller has a new book? https://abella-prover.org/papers/banff2025.pdf 
https://www.lix.polytechnique.fr/Labo/Dale.Miller/ptlp/
Abella https://abella-prover.org/
https://jfr.unibo.it/article/view/4650  Abella: A System for Reasoning about Relational Specification



In [64]:
from dataclasses import dataclass

type Atom = str
type Swap = tuple[Atom, Atom]
type Perm = tuple[Swap, ...] # 

# The inlining chatgpt did made this so short. Quite nice really.
def act(p : Perm, a: Atom) -> Atom:
    for (u,v) in p:
        if a == u:
            a = v
        elif a == v:
            a = u
    return a

id = ()
assert act(id, 'x') == 'x'
xy = (('x', 'y'),)
assert act(xy, 'x') == 'y'
assert act(xy, 'y') == 'x'
assert act(xy, 'z') == 'z'
# compose is just +
yz = (('y', 'z'),)
assert act(xy + yz, 'x') == 'z'
assert act(xy + yz, 'y') == 'x'
def inv(p : Perm) -> Perm:
    return tuple(reversed(p))
assert all(act(inv(xy + yz), act(xy + yz, c)) == c for c in 'xyz')
assert all(act(inv(xy + yz) + xy + yz, c) == c for c in 'xyz')

def is_eq(p : Perm, q : Perm) -> bool:
    support = {c for ab in p + q for c in ab}
    return all(act(p, c) == act(q, c) for c in support)
assert is_eq(id, inv(xy + yz) + xy + yz)
assert is_eq(inv(yz) + inv(xy), inv(xy + yz))


def canon(p : Perm) -> Perm:
    # no. I'm not sure this works at all?
    support = sorted({c for ab in p for c in ab})
    print(support)
    res = []
    for c in support:
        a = act(res, c)
        b = act(p, c)
        if a < b:
            res.append((a,b))
        elif a > b:
            res.append((b,a))
        elif a == b:
            continue
        else:
            raise Exception("impossible")
    return tuple(res)



canon(xy + yz)
canon(yz + xy)




['x', 'y', 'z']
['x', 'y', 'z']


(('x', 'y'), ('x', 'z'))

In [ ]:
def canon(p : Perm) -> Perm:
    support = sorted({c for ab in p for c in ab})
    p2 = {c : act(p, c) for c in support} # convert totalized form into permutation sequence form
    res = []
    for c in support:
        # cycle involving c
        b = act(res, c)
        while b != c:
            b1 = act(res, b)
            res.append((b, b1))
            b = b1

        b = act(p, c):

        while b = act(p, c):
        a = act(perm, c)
        b = act(p, c)
        if a < b:
            rename[a] = b
            perm.append((a,b))
        elif a > b:
            rename[a] = b
            perm.append((b,a))
        elif a == b:

# unify regular

In [ ]:
@dataclass(frozen=True)
class App:
    f : str
    args : tuple['Term', ...]
@dataclass(frozen=True)
class Var:
    name : str
type Term = App | Var
type Subst = list[tuple[str, Term]] # triangular x -> t

def subst(t : Term, v : str, u : Term) -> Term:
    match t:
        case Var(x):
            if x == v:
                return u
            else:
                return t
        case App(f, args):
            return App(f, tuple(subst(arg, v, u) for arg in args))

def apply(s : Subst, t : Term) -> Term:
    match t:
        case Var(x):
            for (y,u) in s:
                if x == y:
                    return apply(s, u) # could give tail of s here.
            return t
        case App(f, args):
            return App(f, tuple(apply(s, arg) for arg in args))
        
def unify(t, u, subst=None) -> Subst | None:
    if subst is None:
        subst = []
    else:
        subst = subst.copy()
    eqs = [(t,u)]
    while eqs:
        t, u = eqs.pop()
        t, u = apply(subst, t), apply(subst, u)
        match t, u:
            case Var(x), Var(y) if x == y:
                continue
            case Var(x), _:
                # occurs_check
                subst.append((x,u))
            case _, Var(y):
                subst.append((y,t))
            case App(f1, args1), App(f2, args2):
                if f1 == f2 and len(args1) == len(args2):
                    eqs.extend(zip(reversed(args1), reversed(args2)))
                else:
                    return None
            case _:
                raise ValueError("Unsupported term structure", t, u)
    return subst
            
f = lambda x: App("f", (x,))
g = lambda x,y: App("g", (x,y))
X = Var("X")
Y = Var("Y")
a = App("a", ())
b = App("b", ())

assert unify(f(X), f(a)) == [('X', a)]
assert unify(f(X), f(Y)) == [('X', Y)]
assert unify(f(X), f(f(X))) == [('X', f(X))]
assert unify(f(X), f(f(Y))) == [('X', f(Y))]
assert unify(g(X,Y), g(a,b)) == [('X', a), ('Y', b)]
assert unify(g(X,Y), g(a,X)) == [('X', a), ('Y', a)]
assert unify(g(X,Y), g(X,X)) == [('Y', X)]
assert unify(a, b) is None
assert unify(X, f(X)) == [('X', f(X))]
assert unify(g(a,b), g(X,X)) is None
    

In [45]:
@dataclass(frozen=True)
class App:
    f : str
    args : tuple['Term', ...]
@dataclass(frozen=True)
class Var:
    name : str
type Term = App | Var
type Subst = dict[str, Term] # triangular x -> t

def apply(s : Subst, t : Term) -> Term:
    match t:
        case Var(x):
            u = s.get(x)
            if u is None:
                return t
            else:
                return apply(s, u)
        case App(f, args):
            return App(f, tuple(apply(s, arg) for arg in args))
        
def unify(t, u, subst=None) -> Subst | None:
    if subst is None:
        subst = {}
    else:
        subst = subst.copy()
    eqs = [(t,u)]
    while eqs:
        t, u = eqs.pop()
        t, u = apply(subst, t), apply(subst, u)
        match t, u:
            case Var(x), Var(y) if x == y:
                continue
            case Var(x), _:
                # occurs_check
                subst[x] = u
            case _, Var(y):
                subst[y] = t
            case App(f1, args1), App(f2, args2):
                if f1 == f2 and len(args1) == len(args2):
                    eqs.extend(zip(reversed(args1), reversed(args2)))
                else:
                    return None
            case _:
                raise ValueError("Unsupported term structure", t, u)
    return subst
            
f = lambda x: App("f", (x,))
g = lambda x,y: App("g", (x,y))
X = Var("X")
Y = Var("Y")
a = App("a", ())
b = App("b", ())

assert unify(f(X), f(a)) == {'X' :  a}
assert unify(f(X), f(Y)) == {'X' : Y}
assert unify(f(X), f(f(X))) == {'X' : f(X)}
assert unify(f(X), f(f(Y))) == {'X' : f(Y)}
assert unify(g(X,Y), g(a,b)) == {'X' : a, 'Y' : b}
assert unify(g(X,Y), g(a,X)) == {'X' : a, 'Y' : a}
assert unify(g(X,Y), g(X,X)) == {'Y' : X}
assert unify(a, b) is None
assert unify(X, f(X)) == {'X' : f(X)}
assert unify(g(a,b), g(X,X)) is None

    

# ground alpha eq
It's always smart to think about ground first

In [66]:
@dataclass(frozen=True)
class App:
    f : str
    args : tuple['Term', ...]
type Perm = tuple[tuple[str, str], ...]
@dataclass(frozen=True)
class Atom:
    name : str

type Term = App | Var | Atom | Perm
type Subst = dict[str, Term] # triangular x -> t

def act_atom(p : Perm, a: str) -> str:
    for (u,v) in p:
        if a == u:
            a = v
        elif a == v:
            a = u
    return a

def alpha_eq(t, u) -> Perm | None: # permutation relevant equality
    eqs = [(t,u)]
    perm = [] # id
    #support = set()
    rename = {}
    # perm and rename vs rename and inv_rename ?
    # what's the point of having
    while eqs:
        t, u = eqs.pop()
        match t, u:
            case Atom(a), Atom(b):
                if a in rename:
                    if rename[a] != b:
                        return None
                    else:
                        continue
                else:
                    rename[a] = b # This is really unify but only var to var unify
                    if act_atom(perm, a) != b:
                        perm.append((a,b))
                """
                if a in support or b in support:
                    if act_atom(perm, a) != b:
                        return None
                else:
                    support.add(a)
                    support.add(b)
                    perm.append((a,b))
                """
            case App(f1, args1), App(f2, args2):
                if f1 == f2 and len(args1) == len(args2):
                    eqs.extend(zip(reversed(args1), reversed(args2)))
                else:
                    return None
            case _,_:
                raise ValueError("Unsupported term structure", t, u)
    #return rename
    return tuple(perm)
a = Atom('a')
b = Atom('b')
g = lambda x,y: App("g", (x,y))
f = lambda x: App("f", (x,))


assert alpha_eq(Atom('x'), Atom('x')) == ()
assert alpha_eq(Atom('x'), Atom('y')) == (("x", "y"),)
assert alpha_eq(f(Atom('x')), f(Atom('x'))) == ()
assert alpha_eq(f(Atom('x')), f(Atom('y'))) == (("x", "y"),)
assert alpha_eq(g(Atom('x'), Atom('y')), g(Atom('a'), Atom('b'))) == (("x", "a"), ("y", "b"))
# Hmm. Should I assume atoms in the two term are distinct?
assert alpha_eq(g(Atom('x'), Atom('y')), g(Atom('a'), Atom('x'))) == (("x", "a"), ("y", "x"))
assert alpha_eq(g(Atom('x'), Atom('y')), g(Atom('y'), Atom('x'))) == (("x", "y"),)



In [ ]:
@dataclass(frozen=True)
class Var:
    p : Perm
    bad_atoms : frozenset[str]
    name : str

In [ ]:
def act_term(p : Perm, a: Term) -> Term:
    match a:
        case Atom(name):
            return Atom(act_atom(p, name))
        case Var(p1, x):
            return Var(p1 + p, x)
        case App(f, args):
            return App(f, tuple(act_term(p, arg) for arg in args))    

def apply(s : Subst, t : Term) -> Term:
    match t:
        case Var(p, x):
            u = s.get(x)
            if u is None:
                return t
            else:
                return apply(s, act_term(inv(p), u))
        case Atom(name):
            return t
        case App(f, args):
            return App(f, tuple(apply(s, arg) for arg in args))

In [ ]:
def unify(t, u, subst=None) -> Subst | None:
    if subst is None:
        subst = {}
    else:
        subst = subst.copy()
    symmettries = [] #?
    eqs = [(t,u)]
    while eqs:
        t, u = eqs.pop()
        t, u = apply(subst, t), apply(subst, u)
        match t, u:
            case Var(px, x), Var(py, y) if x == y:
                if is_eq(px, py):
                    continue
                else:
                    return None # Hmm. But here we might say we can return a set of substitutions
            case Var(x), _:
                # occurs_check
                subst[x] = u
            case _, Var(y):
                subst[y] = t
            case App(f1, args1), App(f2, args2):
                if f1 == f2 and len(args1) == len(args2):
                    eqs.extend(zip(reversed(args1), reversed(args2)))
                else:
                    return None
            case _:
                raise ValueError("Unsupported term structure", t, u)
    return subst
            
f = lambda x: App("f", (x,))
g = lambda x,y: App("g", (x,y))
X = Var("X")
Y = Var("Y")
a = App("a", ())
b = App("b", ())

assert unify(f(X), f(a)) == {'X' :  a}
assert unify(f(X), f(Y)) == {'X' : Y}
assert unify(f(X), f(f(X))) == {'X' : f(X)}
assert unify(f(X), f(f(Y))) == {'X' : f(Y)}
assert unify(g(X,Y), g(a,b)) == {'X' : a, 'Y' : b}
assert unify(g(X,Y), g(a,X)) == {'X' : a, 'Y' : a}
assert unify(g(X,Y), g(X,X)) == {'Y' : X}
assert unify(a, b) is None
assert unify(X, f(X)) == {'X' : f(X)}
assert unify(g(a,b), g(X,X)) is None

    

Oh, I think Susp can float if we like. It doesn't have to be around variables.
Then push it down. 

In [13]:
# chatgpt did this
from dataclasses import dataclass
from typing import Tuple, Dict, List, Set, Union

Atom = str
Var  = str
Perm = Tuple[Tuple[Atom, Atom], ...]  # list of swaps (a b)

@dataclass(frozen=True) class A: a: Atom                 # atom
@dataclass(frozen=True) class V: x: Var                  # variable
@dataclass(frozen=True) class Abs: a: Atom; t: "Term"    # <a>t
@dataclass(frozen=True) class App: f: str; args: Tuple["Term", ...]
@dataclass(frozen=True) class Susp: p: Perm; x: Var      # p · X

Term = Union[A, V, Abs, App, Susp]
Subst = Dict[Var, Term]
Fresh = Set[Tuple[Atom, Term]]  # (a # t)

def swap(p: Perm, a: Atom) -> Atom:
    for (u,v) in p:
        if a == u: a = v
        elif a == v: a = u
    return a

def act(p: Perm, t: Term) -> Term:
    if isinstance(t, A):   return A(swap(p, t.a))
    if isinstance(t, V):   return Susp(p, t.x)  # delay: turn into suspension
    if isinstance(t, Susp):return Susp(p + t.p, t.x)  # compose (sketchy but ok here)
    if isinstance(t, Abs): return Abs(swap(p, t.a), act(p, t.t))
    if isinstance(t, App): return App(t.f, tuple(act(p,u) for u in t.args))
    raise TypeError(t)

def apply(s: Subst, t: Term) -> Term:
    if isinstance(t, V) and t.x in s: return apply(s, s[t.x])
    if isinstance(t, Susp) and t.x in s: return act(t.p, apply(s, s[t.x]))
    if isinstance(t, Abs): return Abs(t.a, apply(s, t.t))
    if isinstance(t, App): return App(t.f, tuple(apply(s,u) for u in t.args))
    return t

def occurs(x: Var, t: Term) -> bool:
    if isinstance(t, V): return t.x == x
    if isinstance(t, Susp): return t.x == x
    if isinstance(t, Abs): return occurs(x, t.t)
    if isinstance(t, App): return any(occurs(x,u) for u in t.args)
    return False

def unify(eqs: List[Tuple[Term,Term]], fr: Fresh) -> Tuple[Subst, Fresh]:
    s: Subst = {}
    while eqs:
        t,u = eqs.pop()
        t,u = apply(s,t), apply(s,u)
        if t == u: continue

        # variable / suspension cases
        if isinstance(t, V) and not occurs(t.x, u):
            s[t.x] = u; continue
        if isinstance(u, V) and not occurs(u.x, t):
            s[u.x] = t; continue
        if isinstance(t, Susp) and not occurs(t.x, u):
            s[t.x] = act(t.p, u); continue
        if isinstance(u, Susp) and not occurs(u.x, t):
            s[u.x] = act(u.p, t); continue

        # structure
        if isinstance(t, App) and isinstance(u, App) and t.f == u.f and len(t.args)==len(u.args):
            eqs.extend(list(zip(t.args, u.args))); continue

        # binder: <a>t = <b>u  ==>  t = (a b)·u    (and maybe freshness side-conditions in full algo)
        if isinstance(t, Abs) and isinstance(u, Abs):
            eqs.append((t.t, act(((t.a,u.a),), u.t))); continue

        # freshness: a # <b>t  ==>  (a==b) or a # t   (very simplified)
        # full algorithm propagates freshness through structure + suspensions carefully.
        raise ValueError("clash / need full nominal+freshness rules here")

    return s, {(a, apply(s,t)) for (a,t) in fr}


def canon(s : Perm):

# the spray of unino find thing


SyntaxError: invalid syntax (1197798417.py, line 9)

A constructive approach to nominal

In [ ]:
from kdrag.all import *
Var = kd.Struct("Var", smt.IntSort())
i,j,k  = smt.Ints("i j k")
swap = kd.Function("swap", [i,j,k], smt.If(i == j, k, smt.If(i == k, j, i)))

def Equivariant(P : smt.BoolSort): # Or internalized as predicate?
    ...


# knuckle old
How to do nominal knuckeldragger? Semantic model?

Could help me understand slotted

https://www.cl.cam.ac.uk/~amp12/papers/nomlfo/nomlfo-draft.pdf

https://www.cl.cam.ac.uk/~amp12/talks/tacs01.pdf talk

No, maybe Tuple(Prim, AtomSet)

In [ ]:
# oof this is unsopund in the presence on FreshVars. Huh. Subtle or obvious?
def FreshAtom(xs, prefix=None):
    """
    Axiom schema for there are always new constants.
    >>> FreshAtom([a, b], prefix="fred")
    (fred!..., |= Distinct(fred!..., a, b))
    """
    # TODO: might also need a Seq form of
    z = smt.FreshConst(Atom, prefix=prefix)
    return z, kd.axiom(smt.Distinct(z, *xs))

# But this is the same thing. Uh oh.
def DistinctN(n : int):
    xs = [smt.Const(f"x{i}", Atom) for i in range(n)]
    z = smt.Const("z", Atom)
    return kd.axiom(smt.ForAll([xs], smt.Exists([z], Distinct(z, *xs))))

# Is FreshVar unsound?
# I can prove something exists
# forall exists does not imply exists forall

Atom = smt.DeclareSort("Atom")


x,y,z = smt.Consts("x y z", Atom)
kd.define("swap", [x,y,z], 
    smt.If(x == z, y,
    smt.If(y == z, x,
    z))
)

swap = kd.notation.SortDispatch(name="swap")
free = kd.notation.SortDispatch(name="free")

swap.register()

def NomSort(s : smt.SortRef) -> smt.SortRef:
    dt = kd.Inductive("Nom_" + s.name(), [s])
    dt.declare("Atom", ("atom", Atom)) # Set(Atom) ? Var?
    dt.declare("Value", ("prim", s))
    dt = dt.create()
    v = smt.Const("v", dt)
    swap.define([s,x,y], 
        smt.If(dt.is_Atom(v), dt.Atom(swap(x,y,dt.prim(v))),
        dt.Value(v))
    )
    # v # t
    distinct.define([s,x],
        smt.If(dt.is_Atom(v), v != dt.atom(x),
        smt.BoolVal(False)
    ))

    return dt.create(), swap, free



#  ??????.  phi(x,y,z) ?    (Bool, Set(Atom)) ? Perm -> Bool ?  
Bool = NormSort(smt.BoolSort())

f = smt.ArrayConst("f", Atom, Atom)
is_iso = kd.define("is_iso", [f], 
    smt.ForAll([x],
        smt.Exists([y],
            f[x] == y
        )
    )
)
compact = kd.define("compact", [f],
    smt.Exists([seq],
        f[a] == smt.In(seq, a)
    )
    smt.ForAll([x,y,z],
        smt.Implies(f[x] == f[y], x == y)
    )
)

perm_inv = kd.define("perm_inv", [a],
    smt.ForAll([x,y],
        smt.Implies(free(a,x) == free(a,y), x == y)
    )
)


NomVar = smt.DeclareSort("NomVar")

def Nominal(S : smt.SortRef):
    dt = kd.Inductive("Nom_" + S.name())
    dt.declare("Nom", ("var", NomVar))
    dt.declare("Const", ("val", S))
    return dt.create()

# not good enoguh for recursive trees.

def NomDT(name):
    dt = kd.Inductive("name")
    dt.declare("Var", ("var", NomVar))
    return dt

# but also it has a different 